# 11. Model Improvement

---

## Objective

Improve the baseline Logistic Regression model (AUC=0.828) to meet all three business targets:

| Metric | Current | Target | Minimum |
|--------|---------|--------|---------|
| ROC-AUC | 0.828 | >= 0.85 | >= 0.80 |
| Recall | 86.1% | >= 80% | >= 70% |
| Precision | 46.3% | >= 50% | >= 40% |

**Strategy:**
1. Feature engineering (14 new features)
2. Test XGBoost, LightGBM, Random Forest (tuned), Gradient Boosting (tuned)
3. Ensemble if multiple models beat baseline
4. Threshold re-optimization on best model

**Rules:**
- No data leakage (statistics computed on train only)
- Same CV strategy as previous notebooks (StratifiedKFold 5, random_state=42)
- Test set touched ONCE at the very end
- Overfitting guards: Train-Val AUC gap < 0.05, CV std < 0.03

---

## 1. Setup & Data Loading

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
from pathlib import Path

from sklearn.model_selection import (
    RandomizedSearchCV, GridSearchCV, StratifiedKFold,
    cross_validate, cross_val_score
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    VotingClassifier, StackingClassifier
)
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    roc_auc_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve
)

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("Libraries loaded.")

Libraries loaded.


In [4]:
PROJECT_ROOT = Path.cwd().parent
DATA_PATH = PROJECT_ROOT / "data" / "processed"
MODELS_PATH = PROJECT_ROOT / "models"

# Load RAW splits (before any encoding/scaling)
X_train = pd.read_csv(DATA_PATH / "X_train.csv")
X_val = pd.read_csv(DATA_PATH / "X_val.csv")
X_test = pd.read_csv(DATA_PATH / "X_test.csv")

y_train = pd.read_csv(DATA_PATH / "y_train.csv").squeeze()
y_val = pd.read_csv(DATA_PATH / "y_val.csv").squeeze()
y_test = pd.read_csv(DATA_PATH / "y_test.csv").squeeze()

print(f"X_train: {X_train.shape}")
print(f"X_val:   {X_val.shape}")
print(f"X_test:  {X_test.shape}")
print(f"\nChurn rate - Train: {y_train.mean():.2%}, Val: {y_val.mean():.2%}, Test: {y_test.mean():.2%}")
print(f"\nColumns: {list(X_train.columns)}")

X_train: (4225, 19)
X_val:   (1409, 19)
X_test:  (1409, 19)

Churn rate - Train: 26.53%, Val: 26.54%, Test: 26.54%

Columns: ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges']


---

## 2. Feature Engineering

Create 14 new features from raw data. All statistics (medians, etc.) are computed on training set only.

In [5]:
def engineer_features(df, train_medians=None, fit=False):
    """
    Create engineered features from raw data.
    
    Parameters
    ----------
    df : pd.DataFrame - Raw features
    train_medians : dict - Median MonthlyCharges per InternetService (from training set)
    fit : bool - If True, compute and return train_medians
    
    Returns
    -------
    df_eng : pd.DataFrame - DataFrame with new features added
    train_medians : dict - (only if fit=True)
    """
    df_eng = df.copy()
    
    # --- Ensure TotalCharges is numeric ---
    df_eng['TotalCharges'] = pd.to_numeric(df_eng['TotalCharges'], errors='coerce').fillna(0)
    
    # === 2.1 Ratio / Derived Numerical Features (5) ===
    safe_tenure = df_eng['tenure'].clip(lower=1)
    
    df_eng['avg_monthly_spend'] = df_eng['TotalCharges'] / safe_tenure
    df_eng['charge_tenure_ratio'] = df_eng['MonthlyCharges'] / safe_tenure
    df_eng['charge_deviation'] = df_eng['MonthlyCharges'] - df_eng['avg_monthly_spend']
    df_eng['expected_lifetime_value'] = df_eng['MonthlyCharges'] * df_eng['tenure']
    
    # Overpay indicator (median from training set only)
    if fit:
        train_medians = df_eng.groupby('InternetService')['MonthlyCharges'].median().to_dict()
    
    df_eng['overpay_indicator'] = (
        df_eng['MonthlyCharges'] 
        - df_eng['InternetService'].map(train_medians)
    )
    
    # === 2.2 Tenure Binning (1) ===
    bins = [0, 6, 12, 24, 48, 72]
    labels = [0, 1, 2, 3, 4]  # new, early, developing, established, loyal
    df_eng['tenure_group'] = pd.cut(
        df_eng['tenure'], bins=bins, labels=labels, include_lowest=True
    ).astype(float)
    
    # === 2.3 Service Count Aggregation (2) ===
    support_cols = ['OnlineSecurity', 'TechSupport', 'OnlineBackup', 'DeviceProtection']
    streaming_cols = ['StreamingTV', 'StreamingMovies']
    
    df_eng['num_support_services'] = sum(
        (df_eng[col] == 'Yes').astype(int) for col in support_cols
    )
    df_eng['num_streaming_services'] = sum(
        (df_eng[col] == 'Yes').astype(int) for col in streaming_cols
    )
    
    # === 2.4 Interaction Features (4) ===
    df_eng['is_mtm_fiber'] = (
        (df_eng['Contract'] == 'Month-to-month') & 
        (df_eng['InternetService'] == 'Fiber optic')
    ).astype(int)
    
    df_eng['is_mtm_no_support'] = (
        (df_eng['Contract'] == 'Month-to-month') & 
        (df_eng['num_support_services'] == 0)
    ).astype(int)
    
    df_eng['is_echeck_mtm'] = (
        (df_eng['PaymentMethod'] == 'Electronic check') & 
        (df_eng['Contract'] == 'Month-to-month')
    ).astype(int)
    
    contract_ord = df_eng['Contract'].map({
        'Month-to-month': 0, 'One year': 1, 'Two year': 2
    })
    df_eng['tenure_x_contract'] = df_eng['tenure'] * contract_ord
    
    # === 2.5 Binary Simplification (2) ===
    df_eng['is_auto_pay'] = df_eng['PaymentMethod'].isin([
        'Bank transfer (automatic)', 'Credit card (automatic)'
    ]).astype(int)
    
    df_eng['has_internet'] = (df_eng['InternetService'] != 'No').astype(int)
    
    if fit:
        return df_eng, train_medians
    return df_eng

In [6]:
# Apply feature engineering (fit on train, transform val/test)
X_train_eng, train_medians = engineer_features(X_train, fit=True)
X_val_eng = engineer_features(X_val, train_medians=train_medians)
X_test_eng = engineer_features(X_test, train_medians=train_medians)

new_features = [
    'avg_monthly_spend', 'charge_tenure_ratio', 'charge_deviation',
    'expected_lifetime_value', 'overpay_indicator', 'tenure_group',
    'num_support_services', 'num_streaming_services',
    'is_mtm_fiber', 'is_mtm_no_support', 'is_echeck_mtm',
    'tenure_x_contract', 'is_auto_pay', 'has_internet'
]

print(f"Original features: {X_train.shape[1]}")
print(f"After engineering: {X_train_eng.shape[1]}")
print(f"New features ({len(new_features)}): {new_features}")
print(f"\nSample of new features (first 5 rows):")
X_train_eng[new_features].head()

Original features: 19
After engineering: 33
New features (14): ['avg_monthly_spend', 'charge_tenure_ratio', 'charge_deviation', 'expected_lifetime_value', 'overpay_indicator', 'tenure_group', 'num_support_services', 'num_streaming_services', 'is_mtm_fiber', 'is_mtm_no_support', 'is_echeck_mtm', 'tenure_x_contract', 'is_auto_pay', 'has_internet']

Sample of new features (first 5 rows):


,avg_monthly_spend,charge_tenure_ratio,charge_deviation,expected_lifetime_value,overpay_indicator,tenure_group,num_support_services,num_streaming_services,is_mtm_fiber,is_mtm_no_support,is_echeck_mtm,tenure_x_contract,is_auto_pay,has_internet
0,60.870455,0.926515,0.279545,4035.90,4.90,4.0,3,0,0,0,0,132,1,1
1,83.987692,1.305385,0.862308,5515.25,28.60,4.0,4,2,0,0,0,130,1,1
2,18.811111,0.282639,1.538889,1465.20,0.20,4.0,0,0,0,0,0,144,0,0
3,72.841912,1.072794,0.108088,4960.60,16.70,4.0,2,2,0,0,0,136,1,1
4,36.020833,2.958333,-0.520833,426.00,-20.75,1.0,2,0,0,0,0,0,0,1


In [7]:
# Quick sanity check: correlation of new features with churn
train_with_target = X_train_eng[new_features].copy()
train_with_target['Churn'] = y_train.values

correlations = train_with_target.corr()['Churn'].drop('Churn').sort_values(key=abs, ascending=False)
print("Correlation of new features with Churn:")
print(correlations.to_string())

Correlation of new features with Churn:
is_mtm_fiber               0.424005
is_echeck_mtm              0.377394
charge_tenure_ratio        0.374263
tenure_group              -0.356248
tenure_x_contract         -0.343074
is_mtm_no_support          0.254760
overpay_indicator         -0.242903
has_internet               0.231284
is_auto_pay               -0.218645
avg_monthly_spend          0.197701
expected_lifetime_value   -0.191795
num_support_services      -0.169072
num_streaming_services     0.074541
charge_deviation          -0.012696


---

## 3. Feature Set Configurations

| Config | Description | For |
|--------|-------------|-----|
| A | 7 current features + 14 engineered | LR |
| B | All 19 raw + 14 engineered | Tree-based models |
| C | 11 raw + 14 engineered | Compromise |

In [8]:
# Define feature groups
FEATURES_CURRENT_7 = [
    'tenure', 'MonthlyCharges', 'Contract', 'InternetService',
    'OnlineSecurity', 'TechSupport', 'PaymentMethod'
]

FEATURES_ALL_RAW = [
    'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
    'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
    'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
    'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
    'MonthlyCharges', 'TotalCharges'
]

FEATURES_MODERATE = [
    'tenure', 'MonthlyCharges', 'TotalCharges', 'SeniorCitizen',
    'Dependents', 'PaperlessBilling', 'Contract', 'InternetService',
    'OnlineSecurity', 'TechSupport', 'PaymentMethod'
]

ENGINEERED = new_features

# Build config sets
config_A = FEATURES_CURRENT_7 + ENGINEERED
config_B = FEATURES_ALL_RAW + ENGINEERED
config_C = FEATURES_MODERATE + ENGINEERED

# Remove duplicates while preserving order
config_A = list(dict.fromkeys(config_A))
config_B = list(dict.fromkeys(config_B))
config_C = list(dict.fromkeys(config_C))

print(f"Config A: {len(config_A)} features")
print(f"Config B: {len(config_B)} features")
print(f"Config C: {len(config_C)} features")

Config A: 21 features
Config B: 33 features
Config C: 25 features


---

## 4. Preprocessing Pipelines

Two pipelines:
- **Pipeline-linear**: StandardScaler + OneHotEncoder (for LR)
- **Pipeline-tree**: No scaling + OrdinalEncoder (for tree-based models)

In [9]:
def identify_column_types(df, feature_list):
    """Identify numerical and categorical columns in the feature list."""
    num_cols = []
    cat_cols = []
    for col in feature_list:
        if df[col].dtype in ['float64', 'int64', 'float32', 'int32']:
            num_cols.append(col)
        else:
            cat_cols.append(col)
    return num_cols, cat_cols


def build_linear_pipeline(df, feature_list):
    """Build preprocessing pipeline for linear models (StandardScaler + OneHotEncoder)."""
    num_cols, cat_cols = identify_column_types(df, feature_list)
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_cols),
            ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='infrequent_if_exist'), cat_cols)
        ],
        remainder='drop'
    )
    return preprocessor, num_cols, cat_cols


def build_tree_pipeline(df, feature_list):
    """Build preprocessing pipeline for tree-based models (OrdinalEncoder, no scaling)."""
    num_cols, cat_cols = identify_column_types(df, feature_list)
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', num_cols),
            ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_cols)
        ],
        remainder='drop'
    )
    return preprocessor, num_cols, cat_cols


print("Pipeline builders defined.")

Pipeline builders defined.


In [10]:
# Build and fit preprocessing for Config B (tree-based, full features)
tree_preprocessor_B, num_B, cat_B = build_tree_pipeline(X_train_eng, config_B)
tree_preprocessor_B.fit(X_train_eng[config_B])

X_train_tree_B = tree_preprocessor_B.transform(X_train_eng[config_B])
X_val_tree_B = tree_preprocessor_B.transform(X_val_eng[config_B])
X_test_tree_B = tree_preprocessor_B.transform(X_test_eng[config_B])

print(f"Config B (tree): {X_train_tree_B.shape[1]} features after encoding")
print(f"  Numerical: {len(num_B)}, Categorical: {len(cat_B)}")

# Build and fit preprocessing for Config A (linear, minimal)
linear_preprocessor_A, num_A, cat_A = build_linear_pipeline(X_train_eng, config_A)
linear_preprocessor_A.fit(X_train_eng[config_A])

X_train_lin_A = linear_preprocessor_A.transform(X_train_eng[config_A])
X_val_lin_A = linear_preprocessor_A.transform(X_val_eng[config_A])
X_test_lin_A = linear_preprocessor_A.transform(X_test_eng[config_A])

print(f"\nConfig A (linear): {X_train_lin_A.shape[1]} features after encoding")

# Build and fit preprocessing for Config C (linear, moderate)
linear_preprocessor_C, num_C, cat_C = build_linear_pipeline(X_train_eng, config_C)
linear_preprocessor_C.fit(X_train_eng[config_C])

X_train_lin_C = linear_preprocessor_C.transform(X_train_eng[config_C])
X_val_lin_C = linear_preprocessor_C.transform(X_val_eng[config_C])
X_test_lin_C = linear_preprocessor_C.transform(X_test_eng[config_C])

print(f"Config C (linear): {X_train_lin_C.shape[1]} features after encoding")

Config B (tree): 33 features after encoding
  Numerical: 18, Categorical: 15

Config A (linear): 27 features after encoding
Config C (linear): 31 features after encoding


---

## 5. Cross-Validation Setup

In [11]:
# Same CV strategy as all previous notebooks
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

scoring = {
    'roc_auc': 'roc_auc',
    'recall': 'recall',
    'precision': 'precision',
    'f1': 'f1'
}

# Results storage
all_results = []

def evaluate_model(name, model, X_tr, y_tr, X_v, y_v):
    """Evaluate a model with CV + validation set. Returns results dict."""
    # Cross-validation
    cv_scores = cross_validate(
        model, X_tr, y_tr, cv=cv, scoring=scoring,
        return_train_score=True, n_jobs=-1
    )
    
    # Fit on full training set, evaluate on validation
    model.fit(X_tr, y_tr)
    y_val_proba = model.predict_proba(X_v)[:, 1]
    y_train_proba = model.predict_proba(X_tr)[:, 1]
    
    val_auc = roc_auc_score(y_v, y_val_proba)
    train_auc = roc_auc_score(y_tr, y_train_proba)
    
    result = {
        'Model': name,
        'CV_AUC': cv_scores['test_roc_auc'].mean(),
        'CV_AUC_std': cv_scores['test_roc_auc'].std(),
        'CV_Recall': cv_scores['test_recall'].mean(),
        'CV_Precision': cv_scores['test_precision'].mean(),
        'CV_F1': cv_scores['test_f1'].mean(),
        'Val_AUC': val_auc,
        'Train_AUC': train_auc,
        'Overfit_Gap': train_auc - val_auc
    }
    
    print(f"\n{'='*60}")
    print(f"{name}")
    print(f"{'='*60}")
    print(f"  CV AUC:      {result['CV_AUC']:.4f} +/- {result['CV_AUC_std']:.4f}")
    print(f"  CV Recall:   {result['CV_Recall']:.4f}")
    print(f"  CV Precision:{result['CV_Precision']:.4f}")
    print(f"  Val AUC:     {result['Val_AUC']:.4f}")
    print(f"  Train AUC:   {result['Train_AUC']:.4f}")
    print(f"  Overfit Gap: {result['Overfit_Gap']:.4f} {'⚠️' if result['Overfit_Gap'] > 0.05 else '✓'}")
    
    return result, model

print("Evaluation framework ready.")

Evaluation framework ready.


---

## 6. Baseline Reference: Logistic Regression (Engineered Features)

In [12]:
# LR with Config A (minimal + engineered)
lr_A = LogisticRegression(C=1, penalty='l1', solver='saga', class_weight='balanced',
                          max_iter=1000, random_state=RANDOM_STATE)
result_lr_A, fitted_lr_A = evaluate_model(
    "LR (Config A: 7 raw + engineered)", lr_A,
    X_train_lin_A, y_train, X_val_lin_A, y_val
)
all_results.append(result_lr_A)


LR (Config A: 7 raw + engineered)
  CV AUC:      0.8453 +/- 0.0154
  CV Recall:   0.7743
  CV Precision:0.5213
  Val AUC:     0.8576
  Train AUC:   0.8486
  Overfit Gap: -0.0090 ✓


In [13]:
# LR with Config C (moderate + engineered)
lr_C = LogisticRegression(C=1, penalty='l1', solver='saga', class_weight='balanced',
                          max_iter=1000, random_state=RANDOM_STATE)
result_lr_C, fitted_lr_C = evaluate_model(
    "LR (Config C: 11 raw + engineered)", lr_C,
    X_train_lin_C, y_train, X_val_lin_C, y_val
)
all_results.append(result_lr_C)


LR (Config C: 11 raw + engineered)
  CV AUC:      0.8471 +/- 0.0170
  CV Recall:   0.7895
  CV Precision:0.5301
  Val AUC:     0.8613
  Train AUC:   0.8514
  Overfit Gap: -0.0099 ✓


---

## 7. XGBoost

In [14]:
# Class imbalance ratio
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"scale_pos_weight: {scale_pos_weight:.2f}")

xgb_param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.15],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'min_child_weight': [1, 3, 5, 7],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha': [0, 0.01, 0.1, 1.0],
    'reg_lambda': [0.5, 1.0, 2.0, 5.0],
    'scale_pos_weight': [1, scale_pos_weight]
}

xgb_base = XGBClassifier(
    random_state=RANDOM_STATE,
    eval_metric='auc',
    use_label_encoder=False,
    verbosity=0
)

xgb_search = RandomizedSearchCV(
    xgb_base, xgb_param_dist, n_iter=50,
    scoring='roc_auc', cv=cv, random_state=RANDOM_STATE,
    n_jobs=-1, verbose=1
)

print("\nStarting XGBoost RandomizedSearchCV...")
xgb_search.fit(X_train_tree_B, y_train)

print(f"\nBest CV AUC: {xgb_search.best_score_:.4f}")
print(f"Best params: {xgb_search.best_params_}")

scale_pos_weight: 2.77

Starting XGBoost RandomizedSearchCV...
Fitting 5 folds for each of 50 candidates, totalling 250 fits

Best CV AUC: 0.8478
Best params: {'subsample': 1.0, 'scale_pos_weight': 1, 'reg_lambda': 5.0, 'reg_alpha': 0.01, 'n_estimators': 500, 'min_child_weight': 3, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0.2, 'colsample_bytree': 0.6}


In [15]:
# Evaluate best XGBoost
best_xgb = xgb_search.best_estimator_
result_xgb, fitted_xgb = evaluate_model(
    "XGBoost (Config B)", best_xgb,
    X_train_tree_B, y_train, X_val_tree_B, y_val
)
all_results.append(result_xgb)


XGBoost (Config B)
  CV AUC:      0.8478 +/- 0.0159
  CV Recall:   0.5326
  CV Precision:0.6815
  Val AUC:     0.8607
  Train AUC:   0.8700
  Overfit Gap: 0.0093 ✓


---

## 8. LightGBM

In [16]:
lgbm_param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, -1],
    'num_leaves': [15, 31, 50, 70],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
    'min_child_samples': [10, 20, 30, 50],
    'reg_alpha': [0, 0.01, 0.1, 1.0],
    'reg_lambda': [0, 0.1, 1.0, 5.0],
    'is_unbalance': [True, False],
    'min_split_gain': [0, 0.01, 0.1]
}

lgbm_base = LGBMClassifier(
    random_state=RANDOM_STATE,
    verbosity=-1
)

lgbm_search = RandomizedSearchCV(
    lgbm_base, lgbm_param_dist, n_iter=50,
    scoring='roc_auc', cv=cv, random_state=RANDOM_STATE,
    n_jobs=-1, verbose=1
)

print("Starting LightGBM RandomizedSearchCV...")
lgbm_search.fit(X_train_tree_B, y_train)

print(f"\nBest CV AUC: {lgbm_search.best_score_:.4f}")
print(f"Best params: {lgbm_search.best_params_}")

Starting LightGBM RandomizedSearchCV...
Fitting 5 folds for each of 50 candidates, totalling 250 fits

Best CV AUC: 0.8473
Best params: {'subsample': 1.0, 'reg_lambda': 1.0, 'reg_alpha': 0.1, 'num_leaves': 31, 'n_estimators': 500, 'min_split_gain': 0.01, 'min_child_samples': 10, 'max_depth': 3, 'learning_rate': 0.01, 'is_unbalance': False, 'colsample_bytree': 0.7}


In [17]:
# Evaluate best LightGBM
best_lgbm = lgbm_search.best_estimator_
result_lgbm, fitted_lgbm = evaluate_model(
    "LightGBM (Config B)", best_lgbm,
    X_train_tree_B, y_train, X_val_tree_B, y_val
)
all_results.append(result_lgbm)


LightGBM (Config B)
  CV AUC:      0.8473 +/- 0.0154
  CV Recall:   0.5299
  CV Precision:0.6776
  Val AUC:     0.8598
  Train AUC:   0.8723
  Overfit Gap: 0.0124 ✓


---

## 9. Random Forest (Properly Tuned)

In [18]:
rf_param_dist = {
    'n_estimators': [200, 300, 500, 700],
    'max_depth': [6, 8, 10, 12, None],
    'min_samples_split': [5, 10, 15, 20],
    'min_samples_leaf': [3, 5, 8, 10],
    'max_features': ['sqrt', 'log2', 0.3, 0.5],
    'class_weight': ['balanced', 'balanced_subsample'],
    'max_samples': [0.7, 0.8, 0.9, None]
}

rf_base = RandomForestClassifier(
    random_state=RANDOM_STATE,
    n_jobs=-1
)

rf_search = RandomizedSearchCV(
    rf_base, rf_param_dist, n_iter=40,
    scoring='roc_auc', cv=cv, random_state=RANDOM_STATE,
    n_jobs=-1, verbose=1
)

print("Starting Random Forest RandomizedSearchCV...")
rf_search.fit(X_train_tree_B, y_train)

print(f"\nBest CV AUC: {rf_search.best_score_:.4f}")
print(f"Best params: {rf_search.best_params_}")

Starting Random Forest RandomizedSearchCV...
Fitting 5 folds for each of 40 candidates, totalling 200 fits

Best CV AUC: 0.8482
Best params: {'n_estimators': 300, 'min_samples_split': 15, 'min_samples_leaf': 10, 'max_samples': 0.8, 'max_features': 'sqrt', 'max_depth': 8, 'class_weight': 'balanced_subsample'}


In [ ]:
# Evaluate best Random Forest
best_rf = rf_search.best_estimator_
result_rf, fitted_rf = evaluate_model(
    "Random Forest Tuned (Config B)", best_rf,
    X_train_tree_B, y_train, X_val_tree_B, y_val
)
all_results.append(result_rf)

---

## 10. Gradient Boosting (Properly Tuned)

In [ ]:
gb_param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'min_samples_split': [10, 15, 20],
    'min_samples_leaf': [5, 8, 10],
    'max_features': ['sqrt', 0.3, 0.5]
}

gb_base = GradientBoostingClassifier(random_state=RANDOM_STATE)

# Compute sample weights for class balance
sample_weights = np.where(y_train == 1, scale_pos_weight, 1.0)

gb_search = RandomizedSearchCV(
    gb_base, gb_param_dist, n_iter=40,
    scoring='roc_auc', cv=cv, random_state=RANDOM_STATE,
    n_jobs=-1, verbose=1
)

print("Starting Gradient Boosting RandomizedSearchCV...")
gb_search.fit(X_train_tree_B, y_train, sample_weight=sample_weights)

print(f"\nBest CV AUC: {gb_search.best_score_:.4f}")
print(f"Best params: {gb_search.best_params_}")

In [ ]:
# Evaluate best Gradient Boosting
best_gb = gb_search.best_estimator_
result_gb, fitted_gb = evaluate_model(
    "Gradient Boosting Tuned (Config B)", best_gb,
    X_train_tree_B, y_train, X_val_tree_B, y_val
)
all_results.append(result_gb)

---

## 11. Model Comparison

In [ ]:
# Compile all results
results_df = pd.DataFrame(all_results).sort_values('CV_AUC', ascending=False)

print("=" * 90)
print("MODEL COMPARISON (sorted by CV AUC)")
print("=" * 90)
print(results_df.to_string(index=False, float_format='{:.4f}'.format))
print("=" * 90)

# Highlight baseline comparison
baseline_auc = 0.828
print(f"\nBaseline AUC (LR, 13 features, test): {baseline_auc}")
print(f"\nModels beating baseline CV AUC:")
for _, row in results_df.iterrows():
    if row['CV_AUC'] > baseline_auc:
        print(f"  ✓ {row['Model']}: {row['CV_AUC']:.4f} (+{(row['CV_AUC']-baseline_auc)*100:.1f} pts)")

In [ ]:
# Visual comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# CV AUC comparison
models = results_df['Model'].values
cv_aucs = results_df['CV_AUC'].values
cv_stds = results_df['CV_AUC_std'].values

colors = ['#2ecc71' if a > baseline_auc else '#e74c3c' for a in cv_aucs]

axes[0].barh(models, cv_aucs, xerr=cv_stds, color=colors, alpha=0.7, edgecolor='black')
axes[0].axvline(x=baseline_auc, color='red', linestyle='--', label=f'Baseline ({baseline_auc})')
axes[0].axvline(x=0.85, color='blue', linestyle='--', label='Target (0.85)')
axes[0].set_xlabel('CV ROC-AUC')
axes[0].set_title('CV AUC Comparison', fontweight='bold')
axes[0].legend()

# Overfitting gap
gaps = results_df['Overfit_Gap'].values
gap_colors = ['#e74c3c' if g > 0.05 else '#2ecc71' for g in gaps]
axes[1].barh(models, gaps, color=gap_colors, alpha=0.7, edgecolor='black')
axes[1].axvline(x=0.05, color='red', linestyle='--', label='Overfit threshold (0.05)')
axes[1].set_xlabel('Train - Val AUC Gap')
axes[1].set_title('Overfitting Check', fontweight='bold')
axes[1].legend()

plt.tight_layout()
plt.show()

---

## 12. Ensemble (if applicable)

Build an ensemble only if >= 2 models beat the baseline AUC of 0.828.

In [ ]:
# Check which models beat baseline
winners = results_df[results_df['CV_AUC'] > baseline_auc]
print(f"Models beating baseline: {len(winners)}")

if len(winners) >= 2:
    print("\n→ Proceeding with ensemble strategy.")
    
    # Identify the best tree-based models for ensemble
    # (We use the fitted models from sections 7-10)
    ensemble_candidates = {
        'xgb': fitted_xgb,
        'lgbm': fitted_lgbm,
        'rf': fitted_rf
    }
    
    # === 12.1 Soft Voting ===
    voting = VotingClassifier(
        estimators=list(ensemble_candidates.items()),
        voting='soft'
    )
    result_voting, fitted_voting = evaluate_model(
        "Soft Voting (XGB+LGBM+RF)", voting,
        X_train_tree_B, y_train, X_val_tree_B, y_val
    )
    all_results.append(result_voting)
    
else:
    print("\n→ Fewer than 2 models beat baseline. Skipping ensemble.")

In [ ]:
# === 12.2 Stacking (if applicable) ===
if len(winners) >= 2:
    stacking = StackingClassifier(
        estimators=list(ensemble_candidates.items()),
        final_estimator=LogisticRegression(
            max_iter=1000, class_weight='balanced', random_state=RANDOM_STATE
        ),
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE),
        stack_method='predict_proba',
        passthrough=False
    )
    
    result_stacking, fitted_stacking = evaluate_model(
        "Stacking (XGB+LGBM+RF → LR)", stacking,
        X_train_tree_B, y_train, X_val_tree_B, y_val
    )
    all_results.append(result_stacking)

---

## 13. Final Model Selection

In [ ]:
# Updated comparison with ensembles
final_results_df = pd.DataFrame(all_results).sort_values('CV_AUC', ascending=False)

print("=" * 90)
print("FINAL MODEL RANKING")
print("=" * 90)
print(final_results_df.to_string(index=False, float_format='{:.4f}'.format))
print("=" * 90)

# Select best model (highest Val AUC with Overfit_Gap < 0.05)
safe_models = final_results_df[final_results_df['Overfit_Gap'] < 0.05]
if len(safe_models) > 0:
    best_row = safe_models.iloc[0]  # Already sorted by CV_AUC desc
else:
    best_row = final_results_df.iloc[0]

print(f"\n✓ SELECTED: {best_row['Model']}")
print(f"  CV AUC: {best_row['CV_AUC']:.4f}")
print(f"  Val AUC: {best_row['Val_AUC']:.4f}")
print(f"  Overfit Gap: {best_row['Overfit_Gap']:.4f}")

---

## 14. Threshold Optimization (Validation Set)

Same protocol as notebook 10: sweep thresholds, select one that satisfies Recall >= 80% with best Precision.

In [ ]:
# Get predictions from the selected best model on validation set
# NOTE: Update this cell to use the correct fitted model variable
# based on the selection from section 13

# Map model names to fitted objects and test data
model_registry = {
    'XGBoost (Config B)': (fitted_xgb, X_val_tree_B, X_test_tree_B),
    'LightGBM (Config B)': (fitted_lgbm, X_val_tree_B, X_test_tree_B),
    'Random Forest Tuned (Config B)': (fitted_rf, X_val_tree_B, X_test_tree_B),
    'Gradient Boosting Tuned (Config B)': (fitted_gb, X_val_tree_B, X_test_tree_B),
    'LR (Config A: 7 raw + engineered)': (fitted_lr_A, X_val_lin_A, X_test_lin_A),
    'LR (Config C: 11 raw + engineered)': (fitted_lr_C, X_val_lin_C, X_test_lin_C),
}

# Add ensembles if they exist
if 'fitted_voting' in dir():
    model_registry['Soft Voting (XGB+LGBM+RF)'] = (fitted_voting, X_val_tree_B, X_test_tree_B)
if 'fitted_stacking' in dir():
    model_registry['Stacking (XGB+LGBM+RF → LR)'] = (fitted_stacking, X_val_tree_B, X_test_tree_B)

selected_name = best_row['Model']
selected_model, X_val_selected, X_test_selected = model_registry[selected_name]

y_val_proba = selected_model.predict_proba(X_val_selected)[:, 1]
print(f"Selected model: {selected_name}")
print(f"Val predictions generated: {len(y_val_proba)} samples")

In [ ]:
# Threshold sweep
thresholds = np.arange(0.20, 0.85, 0.05)
threshold_results = []

for thresh in thresholds:
    y_pred = (y_val_proba >= thresh).astype(int)
    cm = confusion_matrix(y_val, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    total_churners = tp + fn
    total_non_churners = tn + fp
    
    threshold_results.append({
        'Threshold': round(thresh, 2),
        'Precision': precision_score(y_val, y_pred, zero_division=0),
        'Recall': recall_score(y_val, y_pred, zero_division=0),
        'F1': f1_score(y_val, y_pred, zero_division=0),
        'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn,
        'Churners_Captured_%': tp / total_churners * 100 if total_churners > 0 else 0,
        'FPR_%': fp / total_non_churners * 100 if total_non_churners > 0 else 0,
        'Total_Flagged': tp + fp
    })

df_thresh = pd.DataFrame(threshold_results).round(3)

print("THRESHOLD ANALYSIS (Validation Set)")
print("=" * 80)
print(df_thresh[['Threshold', 'Precision', 'Recall', 'F1', 'Churners_Captured_%', 'FPR_%', 'Total_Flagged']].to_string(index=False))
print("=" * 80)

In [ ]:
# Select optimal threshold: Recall >= 80% with highest Precision
valid_thresholds = df_thresh[df_thresh['Recall'] >= 0.80]

if len(valid_thresholds) > 0:
    best_thresh_row = valid_thresholds.loc[valid_thresholds['Precision'].idxmax()]
    FINAL_THRESHOLD = best_thresh_row['Threshold']
else:
    # Fallback: best F1
    best_thresh_row = df_thresh.loc[df_thresh['F1'].idxmax()]
    FINAL_THRESHOLD = best_thresh_row['Threshold']

print(f"\n✓ SELECTED THRESHOLD: {FINAL_THRESHOLD}")
print(f"  Recall:    {best_thresh_row['Recall']:.1%}")
print(f"  Precision: {best_thresh_row['Precision']:.1%}")
print(f"  F1:        {best_thresh_row['F1']:.3f}")
print(f"  Flagged:   {int(best_thresh_row['Total_Flagged'])} customers")

---

## 15. Final Test Set Evaluation

**First and only time touching the test set.**

In [ ]:
# Test set predictions
y_test_proba = selected_model.predict_proba(X_test_selected)[:, 1]
y_test_pred = (y_test_proba >= FINAL_THRESHOLD).astype(int)

# Metrics
test_roc_auc = roc_auc_score(y_test, y_test_proba)
test_recall = recall_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)

cm_test = confusion_matrix(y_test, y_test_pred)
tn, fp, fn, tp = cm_test.ravel()

print("=" * 60)
print(f"FINAL TEST SET RESULTS — {selected_name}")
print(f"Threshold: {FINAL_THRESHOLD}")
print("=" * 60)

print(f"\n  ROC-AUC:    {test_roc_auc:.4f}  {'✓' if test_roc_auc >= 0.85 else '✗'} (target >= 0.85)")
print(f"  Recall:     {test_recall:.4f}  {'✓' if test_recall >= 0.80 else '✗'} (target >= 0.80)")
print(f"  Precision:  {test_precision:.4f}  {'✓' if test_precision >= 0.50 else '✗'} (target >= 0.50)")
print(f"  F1 Score:   {test_f1:.4f}")

print(f"\nConfusion Matrix:")
print(f"  TP: {tp} | FP: {fp}")
print(f"  FN: {fn} | TN: {tn}")

total_churners = tp + fn
print(f"\nBusiness KPIs:")
print(f"  Churners captured: {tp}/{total_churners} ({tp/total_churners*100:.1f}%)")
print(f"  Churners missed:   {fn}")
print(f"  False alarms:      {fp}")
print(f"  Total flagged:     {tp+fp}")
print("=" * 60)

In [ ]:
# Comparison with baseline
print("\n" + "=" * 60)
print("IMPROVEMENT vs BASELINE")
print("=" * 60)
print(f"{'Metric':<12} {'Baseline':>10} {'New':>10} {'Delta':>10}")
print("-" * 60)
print(f"{'ROC-AUC':<12} {'0.8282':>10} {test_roc_auc:>10.4f} {(test_roc_auc-0.8282)*100:>+9.1f} pts")
print(f"{'Recall':<12} {'0.8610':>10} {test_recall:>10.4f} {(test_recall-0.8610)*100:>+9.1f} pts")
print(f"{'Precision':<12} {'0.4626':>10} {test_precision:>10.4f} {(test_precision-0.4626)*100:>+9.1f} pts")
print(f"{'F1':<12} {'0.6019':>10} {test_f1:>10.4f} {(test_f1-0.6019)*100:>+9.1f} pts")
print("=" * 60)

In [ ]:
# Confusion matrix visualization
fig, ax = plt.subplots(figsize=(8, 6))

total = cm_test.sum()
labels = np.array([
    [f'{tn}\n({tn/total*100:.1f}%)', f'{fp}\n({fp/total*100:.1f}%)'],
    [f'{fn}\n({fn/total*100:.1f}%)', f'{tp}\n({tp/total*100:.1f}%)']
])

sns.heatmap(cm_test, annot=labels, fmt='', cmap='Blues', ax=ax,
            xticklabels=['Predicted: Stay', 'Predicted: Churn'],
            yticklabels=['Actual: Stay', 'Actual: Churn'])
ax.set_title(f'Final Confusion Matrix — {selected_name} (Threshold={FINAL_THRESHOLD})',
             fontsize=12, fontweight='bold')
ax.set_ylabel('Actual')
ax.set_xlabel('Predicted')
plt.tight_layout()
plt.show()

---

## 16. Save Artifacts

In [ ]:
# Save the best model and preprocessor
joblib.dump(selected_model, MODELS_PATH / "best_model_v2.joblib")
print(f"✓ Model saved: {MODELS_PATH / 'best_model_v2.joblib'}")

# Save the appropriate preprocessor
# Determine which preprocessor was used
if 'Config B' in selected_name or 'Voting' in selected_name or 'Stacking' in selected_name:
    joblib.dump(tree_preprocessor_B, MODELS_PATH / "preprocessor_v2.joblib")
    print(f"✓ Preprocessor saved: tree pipeline (Config B)")
elif 'Config A' in selected_name:
    joblib.dump(linear_preprocessor_A, MODELS_PATH / "preprocessor_v2.joblib")
    print(f"✓ Preprocessor saved: linear pipeline (Config A)")
else:
    joblib.dump(linear_preprocessor_C, MODELS_PATH / "preprocessor_v2.joblib")
    print(f"✓ Preprocessor saved: linear pipeline (Config C)")

# Save train medians for feature engineering
joblib.dump(train_medians, MODELS_PATH / "train_medians.joblib")
print(f"✓ Train medians saved (for feature engineering)")

# Save final test results
final_results = {
    'model_type': selected_name,
    'final_threshold': FINAL_THRESHOLD,
    'test_set_size': len(y_test),
    'roc_auc': test_roc_auc,
    'precision': test_precision,
    'recall': test_recall,
    'f1_score': test_f1,
    'true_positives': int(tp),
    'false_positives': int(fp),
    'true_negatives': int(tn),
    'false_negatives': int(fn),
    'evaluation_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
}

pd.DataFrame([final_results]).to_csv(MODELS_PATH / 'final_test_results_v2.csv', index=False)
print(f"✓ Results saved: {MODELS_PATH / 'final_test_results_v2.csv'}")

# Save full comparison table
final_results_df.to_csv(MODELS_PATH / 'model_comparison_v2.csv', index=False)
print(f"✓ Comparison saved: {MODELS_PATH / 'model_comparison_v2.csv'}")

In [ ]:
print("\n" + "=" * 60)
print("NOTEBOOK COMPLETE")
print("=" * 60)
print(f"\nBest model: {selected_name}")
print(f"Threshold:  {FINAL_THRESHOLD}")
print(f"Test AUC:   {test_roc_auc:.4f}")
print(f"Test Recall:{test_recall:.4f}")
print(f"Test Prec:  {test_precision:.4f}")
print(f"\nArtifacts saved:")
print(f"  • best_model_v2.joblib")
print(f"  • preprocessor_v2.joblib")
print(f"  • train_medians.joblib")
print(f"  • final_test_results_v2.csv")
print(f"  • model_comparison_v2.csv")
print("=" * 60)